1. Read the CSV file provided into a Spark DataFrame.

In [9]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

df = spark.read.csv("/data/titanic.csv", header=True, inferSchema=True)

df.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|gender| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| NULL|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| NULL|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

2. Count the number of males and females who had their Cabin as null.

In [10]:
from pyspark.sql.functions import col
filtered_data = df.filter(col('Cabin').isNull())
grouping = filtered_data.groupBy(df.gender).count().show()

+------+-----+
|gender|count|
+------+-----+
|female|  217|
|  male|  470|
+------+-----+



3. Find the average ages of passengers.


In [11]:
from pyspark.sql.functions import avg

average_age = df.select(avg('Age')).collect()[0][0]
average_age = round(average_age , 0)
print(average_age)

30.0


4. Fill in the missing age values with this average value.

In [19]:
df_filled = df.fillna(average_age, subset=['Age'])
print('data before filling null values in age column:')
df.select(col('Age')).show()



data before filling null values in age column:
+-----------+--------+------+--------------------+------+----+-----+-----+---------------+--------+-----+--------+
|PassengerId|Survived|Pclass|                Name|gender| Age|SibSp|Parch|         Ticket|    Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+---------------+--------+-----+--------+
|          6|       0|     3|    Moran, Mr. James|  male|NULL|    0|    0|         330877|  8.4583| NULL|       Q|
|         18|       1|     2|Williams, Mr. Cha...|  male|NULL|    0|    0|         244373|    13.0| NULL|       S|
|         20|       1|     3|Masselmani, Mrs. ...|female|NULL|    0|    0|           2649|   7.225| NULL|       C|
|         27|       0|     3|Emir, Mr. Farred ...|  male|NULL|    0|    0|           2631|   7.225| NULL|       C|
|         29|       1|     3|"O'Dwyer, Miss. E...|female|NULL|    0|    0|         330959|  7.8792| NULL|       Q|
|         30|       0|     3| Tod

In [24]:
print('data after filling null values in age column:')
df_filled.select(col('Age')).show()

data after filling null values in age column:
+----+
| Age|
+----+
|22.0|
|38.0|
|26.0|
|35.0|
|35.0|
|30.0|
|54.0|
| 2.0|
|27.0|
|14.0|
| 4.0|
|58.0|
|20.0|
|39.0|
|14.0|
|55.0|
| 2.0|
|30.0|
|31.0|
|30.0|
+----+
only showing top 20 rows



5.Save the output to a CSV file in HDFS (depi_folder).

In [27]:
df_filled.write.mode("overwrite").csv("/data/titanic_filled.csv")

6. Count the total number of passengers who survived and those who did not.

In [52]:
from pyspark.sql.functions import when, col
df_filled.groupBy('Survived').count().withColumn('Survived', when(col('Survived') == 1, '3ayesh').otherwise('dead :(')).show()

+--------+-----+
|Survived|count|
+--------+-----+
|  3ayesh|  342|
| dead :(|  549|
+--------+-----+



7. Find the top 5 most common embarkation ports among passengers.

In [55]:
df_filled.groupBy('Embarked').count().orderBy(col('count').desc()).show()
                    

+--------+-----+
|Embarked|count|
+--------+-----+
|       S|  644|
|       C|  168|
|       Q|   77|
|    NULL|    2|
+--------+-----+



8. Calculate the survival rate by passenger class (Pclass)

In [58]:
from pyspark.sql.functions import avg, round

df_filled.groupBy('Pclass').agg(round(avg('Survived'), 2).alias('Survival_Rate')).show()

+------+-------------+
|Pclass|Survival_Rate|
+------+-------------+
|     1|         0.63|
|     3|         0.24|
|     2|         0.47|
+------+-------------+



9. Determine the maximum, minimum, and average fare paid by passengers.

In [64]:
from pyspark.sql.functions import max, min

df_filled.agg(round(avg('Fare'),2).alias('Average_Fare'), round(max('Fare'),2).alias('Max_Fare'), min('Fare').alias('Min_Fare')).show()

+------------+--------+--------+
|Average_Fare|Max_Fare|Min_Fare|
+------------+--------+--------+
|        32.2|  512.33|     0.0|
+------------+--------+--------+



10. Write a Spark job to count the number of passengers in each age group (e.g., 0–18, 19–35, 36–60, 60+).

In [72]:
age_groups = df_filled.withColumn(
    "age_group",
    when(col("Age") <= 18, "0-18")
    .when((col("Age") > 18) & (col("Age") <= 35), "19-35")
    .when((col("Age") > 35) & (col("Age") <= 60), "36-60")
    .otherwise("60+")
)

age_groups.groupBy('age_group').count().show()

+---------+-----+
|age_group|count|
+---------+-----+
|    19-35|  535|
|      60+|   22|
|    36-60|  195|
|     0-18|  139|
+---------+-----+



11. Create a new directory in HDFS called titanic_lab and list its contents.

In [ ]:
hdfs dfs -mkdir /titanic_lab
hdfs dfs -ls /

12. Upload the Titanic dataset from your local machine to the titanic_lab directory in HDFS.

In [ ]:
docker exec namenode hdfs dfs -put /data/titanic.csv /titanic_lab/


13. Use chmod command to change the permissions of the Titanic dataset file to 777 (full permissionsfor all)

In [ ]:

docker exec namenode hdfs dfs -chmod 777 /titanic_lab/titanic.csv


14. Use-cat to display the first 20 lines of the Titanic dataset stored in HDFS.

In [ ]:

docker exec namenode hdfs dfs -cat /titanic_lab/titanic.csv | head -20

15- Move the processed output file from titanic_lab to a new directory in HDFS called titanic_results using hdfs dfs -mv.


In [ ]:
 
docker exec namenode hdfs dfs -mkdir /titanic_results


docker exec namenode hdfs dfs -put /data/titanic_filled.csv /titanic_lab/
docker exec namenode hdfs dfs -mv /titanic_lab/titanic_filled.csv /titanic_results/